In [1]:
import torch
import torchvision
from torchvision.models import resnet50

from kiss.models import vgg16_kiss
from kiss.experiment import Experiment
from kiss.sampler import RandomSampler, KMeansSampler, KMeansPuritySampler, KMeansPurity2Sampler, KMeansDinoSampler, KMeansPurityDinoSampler, OpticsSampler
from kiss.utils.configs import CONFIGS

transform = torchvision.transforms.ToTensor()
dataset_cifar100_tr = torchvision.datasets.CIFAR100(root='../data/cifar100', train=True, download=True, transform=transform)
dataset_cifar100_te = torchvision.datasets.CIFAR100(root='../data/cifar100', train=False, download=True, transform=transform)

dataset_svhn_tr = torchvision.datasets.SVHN(root='../data/svhn', split='train', download=True, transform=transform)
dataset_svhn_te = torchvision.datasets.SVHN(root='../data/svhn', split='test', download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ../data/svhn/train_32x32.mat
Using downloaded and verified file: ../data/svhn/test_32x32.mat


In [2]:
model = resnet50(num_classes=100)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model = model,
    dataset_tr = dataset_cifar100_tr,
    dataset_te = dataset_cifar100_te,
    sampler_cls=OpticsSampler,
    ratio=(0.1, 0.3, 3),
    epochs=10,
    batch_size=512,
    clip=5.0,
    num_clusters=8,
    eqsize=True,
    min_purity=0.01,
    # load_clusters="KMeansPurity2Sampler,CIFAR100,nc:8",
    # save_clusters="KMeansPurity2Sampler,CIFAR100,nc:8",
)
experiment.run("../experiments", "GOODVALID")

Clustering: 100%|██████████| 100/100 [00:48<00:00,  2.08it/s]


Running experiment ResNet!CIFAR100!OpticsSampler
Running run GOODVALID/1
len clusters [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1]
general 10 411 0.1 411 41
cluster sizes 10 {0: 411}
keep clusters: [0]
general 86 406 0.1 406 40
cluster sizes 86 {0: 406}
keep clusters: [0]
general 70 395 0.1 395 39
cluster sizes 70 {0: 395}
keep clusters: [0]
general 26 399 0.1 399 39
cluster sizes 26 {0: 399}
keep clusters: [0]
general 97 403 0.1 403 40
cluster sizes 97 {0: 403}
keep clusters: [0]
general 39 399 0.1 399 39
cluster sizes 39 {0: 399}
keep clusters: [0]
general 60 379 0.1 379 37
cluster sizes 60 {0: 379}
keep clusters: [0]
general 24 386 0.1 386 38
cluster sizes 24 {0: 386}
keep clusters: [0]
general 96 392 0.1 392 39
cluster sizes 96 {0

Validating: 100%|██████████| 20/20 [00:03<00:00,  5.40 batch/s, loss=4.6752]


Best valid loss improved. Current accuracy is 0.78%. Saving checkpoint...
Best valid accuracy improved. Current accuracy is 0.78%. Saving checkpoint...


Validating: 100%|██████████| 20/20 [00:03<00:00,  5.49 batch/s, loss=4.8105]


...

Validating: 100%|██████████| 20/20 [00:03<00:00,  5.45 batch/s, loss=7.6336]


Best valid accuracy improved. Current accuracy is 15.62%. Saving checkpoint...


Validating: 100%|██████████| 20/20 [00:03<00:00,  5.56 batch/s, loss=5.0094]


Best valid accuracy improved. Current accuracy is 16.16%. Saving checkpoint...


Testing: 100%|██████████| 20/20 [00:03<00:00,  5.52 batch/s]
